In [ ]:
# requeriment
#%capture
!pip cache purge
!mkdir -p ~/tmp_pip
!TMPDIR=~/tmp_pip pip install --no-cache-dir neuralforecast statsforecast optuna plotly scikit-learn lightning ipywidgets openpyxl nbformat

# Funções auxiliares

In [1]:
from sklearn.metrics import root_mean_squared_error
from neuralforecast.losses.pytorch import HuberLoss
from pytorch_lightning.loggers import CSVLogger
from neuralforecast import NeuralForecast
from IPython.display import clear_output
from neuralforecast.models import LSTM
from datetime import datetime
from torch.optim import AdamW
from pathlib import Path
import pandas as pd
import numpy as np
import optuna
import shutil
import yaml
import os

In [2]:
# Classe para facilitar o uso de cores no terminal
class CoresTerminal:
    """Contém códigos ANSI para colorir o output no terminal."""
    VERMELHO = '\033[91m'
    VERDE = '\033[92m'
    FIM = '\033[0m'


In [3]:
from torch.optim.lr_scheduler import StepLR
import torch
import time

torch.set_float32_matmul_precision('high')

# VARIÁVEIS GLOBAIS PARA LOGGING DE MÉTRICAS DE TREINAMENTO
score_media_simples = 0.0
score_rmse = 0.0
training_duration = 0.0
tempo_total = 0.0
avg_training_time = 0.0


# ===================================================================
# FUNÇÃO OBJECTIVE (RESPONSABILIDADE: TREINAR E AVALIAR)
# ===================================================================
def objective(trial: optuna.trial.Trial) -> float:
    """
    Treina e avalia um modelo para um conjunto de hiperparâmetros.
    Salva os artefatos do trial em uma pasta temporária.
    """
    global score_media_simples, score_rmse, training_duration, avg_training_time, tempo_total

    string_saidas = f"\nTeste do Modelo: {treino_id}_({ano_teste})\n"
    clear_output()
    print(string_saidas)
    # 1) Hiperparâmetros a serem otimizados
    #steps_options = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
    #encoder_n_layers = trial.suggest_int("encoder_n_layers", 3, 8)
    #encoder_hidden_size = trial.suggest_categorical("encoder_hidden_size", [64, 128, 192, 256])
    #decoder_layers = trial.suggest_int("decoder_layers", 1, 4)
    #decoder_hidden_size = trial.suggest_categorical("decoder_hidden_size", [64, 128, 192, 256])
    #weight_decay = trial.suggest_categorical("weight_decay", [1e-5, 1e-4, 1e-2])
    #steps = trial.suggest_categorical("steps", steps_options)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)

    # Parâmetros fixos
    batch_size = 32
    dropout = 0.3
    steps = 1000
    encoder_n_layers= 4
    encoder_hidden_size= 192
    decoder_layers= 1
    decoder_hidden_size= 64
    weight_decay= 0.0001
    # --- Gerenciamento de Logs por Trial ---
    trial_log_dir = log_dir_base / f"trial_{trial.number}"
    if trial_log_dir.exists():
        shutil.rmtree(trial_log_dir)
    trial_log_dir.mkdir(parents=True, exist_ok=True)
    
    csv_logger = CSVLogger(save_dir=str(trial_log_dir.parent), name=trial_log_dir.name)

    # 2) Instanciação do modelo LSTM
    model = LSTM(
        h=h,
        input_size=input_size,
        batch_size=batch_size,
        scaler_type="revin",
        encoder_dropout=dropout,
        encoder_n_layers=encoder_n_layers,
        encoder_hidden_size=encoder_hidden_size,
        decoder_layers=decoder_layers,
        decoder_hidden_size=decoder_hidden_size,
        futr_exog_list=exog_list,
        learning_rate=learning_rate,
        max_steps=steps,
        loss=HuberLoss(delta=1.0),
        optimizer=AdamW,
        optimizer_kwargs={"weight_decay": weight_decay},
        lr_scheduler=StepLR,
        lr_scheduler_kwargs={"step_size": int(steps * 0.5), "gamma": 0.1},
        logger=csv_logger,
        random_seed=42
    )

    # 3) Treinamento
    nf = NeuralForecast(models=[model], freq="YE")
    
    if trial.number != 0:
        print(f"Resultados da Trial anterior:")
        print(f"  - WMAPE: {score_media_simples:.4f}")
        print(f"  - RMSE:  {score_rmse:.4f}")
        print(f"  - Tempo Total Treino: {training_duration:.2f}s")
        print(f"  - Tempo médio de treino até agora: {avg_training_time:.2f}s\n\n")
        print(f"  - Tempo estimado para conclusão (baseado na média): {(num_trials - trial.number) * avg_training_time / 60:.2f} minutos\n")

    print(f"Iniciando treinamento do Trial {trial.number}...")
    start_time = time.time()
    nf.fit(df=train_ds)
    end_time = time.time()

    # Cálculo do tempo de treinamento
    training_duration = end_time - start_time
    tempo_total += training_duration
    avg_training_time = tempo_total / (trial.number + 1)

    # 4) Avaliação
    combined_df = evaluate_simple_forecast(
        model=nf,
        train_df=train_ds,
        test_df=val_ds,
        string_saidas = string_saidas
    )
    

    # 5) Cálculo da métrica
    actual = combined_df["y"]
    predicted = combined_df["LSTM"]

    # 5) Cálculo das métricas
    if np.isfinite(combined_df["LSTM"]).all():
        score_media_simples = calcular_media_wmape_simples(combined_df)
        score_rmse = root_mean_squared_error(actual, predicted)
        score = score_media_simples
    else:
        # Penalidade para previsões inválidas (NaN, inf)
        score = 1e10 
        print("Previsão inválida encontrada. Atribuindo score de penalidade.")


    # --- SALVAMENTO TEMPORÁRIO DOS ARTEFATOS DO TRIAL ---
    trial_artifact_path = temp_model_dir / f"trial_{trial.number}"
    nf.save(path=str(trial_artifact_path), overwrite=True)
    
    # --- LÓGICA DE ESPERA ATIVA (POLLING) ---
    # CORREÇÃO: Construímos o caminho para version_0 explicitamente, pois sabemos que será sempre este.
    log_version_dir = trial_log_dir / "version_0"
    metrics_path = log_version_dir / "metrics.csv"
    hparams_path = log_version_dir / "hparams.yaml"
    
    file_found = False
    # Tenta encontrar o arquivo por até 10 segundos
    for _ in range(100):
        if metrics_path.exists():
            file_found = True
            break
        time.sleep(0.1)

    # Copia os arquivos de log se eles foram encontrados
    if file_found:
        shutil.copy(metrics_path, trial_artifact_path / "metrics.csv")
        shutil.copy(hparams_path, trial_artifact_path / "hparams.yaml")
    else:
        print(f"AVISO: Arquivo metrics.csv não encontrado em {log_version_dir} para o trial {trial.number}")

    # Anexa informações ao trial para o callback poder acessá-las
    trial.set_user_attr("artifact_path", str(trial_artifact_path))
    
    # Converte a coluna de data para string para garantir que seja serializável em JSON.
    combined_df_serializable = combined_df.copy()
    if 'ds' in combined_df_serializable.columns:
        combined_df_serializable['ds'] = combined_df_serializable['ds'].astype(str)
    trial.set_user_attr("prediction_df", combined_df_serializable.to_dict())
    return score

In [4]:
def teste_modelo(local, csv_dir, dataset, train_ds, val_ds, test_ds, comentario, nome_dataset, string_saidas = """""", h = 1, incluir_treino = False):
    print(f"\nTeste do Modelo: {local}")
    # carregamento do modelo salvo
    model = NeuralForecast.load(path=f"{local}")
    if torch.cuda.is_available():
        print("Forçando configuração de GPU no modelo carregado...")
        for model_inst in model.models:
            # O objeto 'model_inst' é o próprio modelo (LSTM).
            # Precisamos alterar os argumentos que ele usará para criar o Trainer interno.
            if hasattr(model_inst, 'trainer_kwargs'):
                model_inst.trainer_kwargs['accelerator'] = 'gpu'
                model_inst.trainer_kwargs['devices'] = 1
            else:
                # Caso a versão seja diferente e não tenha trainer_kwargs exposto,
                # tentamos criar/injetar (menos comum, mas preventivo)
                model_inst.trainer_kwargs = {'accelerator': 'gpu', 'devices': 1}
    else:
        print("AVISO: GPU não detectada. O predict_insample será lento.")

    # --- Execução da avaliação do dados de teste --- 
    predictions = evaluate_simple_forecast(
        model=model,
        train_df=dataset,
        test_df=test_ds,
        split = "Teste"
    )

    dados_teste = predictions[['unique_id', 'ds', 'y', 'LSTM', 'diferença_%']].copy()
    dados_teste.columns = ['unique_id', 'ds', 'y', 'y_pred', 'diferença_%'] # Renomeando as colunas para o formato esperado
    dados_teste['flag'] = 'teste'
    dados_teste['ds'] = pd.to_datetime(dados_teste['ds']).fillna(dados_teste['ds'])
    dados_teste['ds'] = (pd.to_datetime(dados_teste['ds'], errors='coerce').fillna(pd.Timestamp.now()).dt.strftime('%Y-%m-%dT%H:%M:%S'))


    # --- Execução da avaliação do dados de validação --- 
    if val_ds is not None:
        dados_validacao = evaluate_simple_forecast(
            model=model,
            train_df=dataset,
            test_df=val_ds,
            split = "Validação"
        )
        dados_validacao = dados_validacao[['unique_id', 'ds', 'y', 'LSTM', 'diferença_%']].copy()
        dados_validacao.columns = ['unique_id', 'ds', 'y', 'y_pred', 'diferença_%'] # Renomeando as colunas para o formato esperado
        dados_validacao['flag'] = 'validacao'
        dados_validacao['ds'] = pd.to_datetime(dados_validacao['ds']).fillna(dados_validacao['ds'])
        dados_validacao['ds'] = (pd.to_datetime(dados_validacao['ds'], errors='coerce').fillna(pd.Timestamp.now()).dt.strftime('%Y-%m-%dT%H:%M:%S'))
    
    # --- Execução da avaliação do dados de treino --- 
    dados_treino = dataset[['unique_id', 'ds', 'y']].copy()
    dados_treino['y'] = round(np.expm1(dados_treino['y']),2)
    # identifica o ano de termino do treino (pode ser o inicio da validação ou dos testes)
    if val_ds is not None:
        data_inicio = val_ds['ds'].min() # identifica o periodo de inicio da validação
        print(f"Periodo de inicio da Validação: {data_inicio}\n")
    else:
        data_inicio = dados_teste['ds'].min() # identifica o periodo de inicio dos teste
        print(f"Periodo de inicio dos Testes: {data_inicio}\n")
    dados_treino = dados_treino[dados_treino['ds'] < data_inicio] # pega apenas as datas anteriores a data de inicio
    dados_treino['y_pred'] = "-"  # "- " para previsões no treino, nesse ponto elas não exitem. (preenche com valor padão que sera substituido)


    # Extrai as previsões in-sample do modelo para o conjunto de treino
    if incluir_treino:
        print("Treinamento Iniciando a previsão com o modelo!")
        insample_df = model.predict_insample(step_size=h)
        # Inversão da transformação log1p
        insample_df['y'] = np.expm1(insample_df['y'])
        insample_df['LSTM'] = np.expm1(insample_df['LSTM'])
        # Filtrar para garantir contexto suficiente
        contexto =  hiperparametros['input_size'] # verifica o tanho da janela de contexto.
        # remove os primeiros registros que não tem contexto suficiente para ter uma previsão válida.
        min_cutoff_valido = pd.to_datetime(insample_df['cutoff'].min()) + pd.DateOffset(years=contexto) 
        insample_df = insample_df[insample_df['cutoff'] >= min_cutoff_valido].copy()

        insample_preds = insample_df[['unique_id', 'ds', 'LSTM']].copy()
        insample_preds = insample_preds.rename(columns={'LSTM': 'y_pred_temp'})
        # Faz o merge dos dados de treino com as previsões disponíveis no insample_df
        dados_treino = dados_treino.merge(insample_preds, on=['unique_id', 'ds'], how='left')
        # Atualiza a coluna y_pred: onde tiver previsão do insample_df usa ela, senão mantém o "-"
        dados_treino['y_pred'] = dados_treino['y_pred_temp'].combine_first(dados_treino['y_pred'])
        # Remove a coluna temporária
        dados_treino = dados_treino.drop(columns=['y_pred_temp'])

        dados_treino['diferença_%'] = "-"  # "-" para a diferença percentual entre predito e real, nesse ponto elas não exitem.
        dados_treino['flag'] = 'treino'
        dados_treino['ds'] = (pd.to_datetime(dados_treino['ds'], errors='coerce').fillna(pd.Timestamp.now()).dt.strftime('%Y-%m-%dT%H:%M:%S'))

        dados_treino['diferença_%'] = dados_treino.apply(calcula_diferenca_pct, axis=1)
        dados_treino['flag'] = 'treino'
        dados_treino['ds'] = (pd.to_datetime(dados_treino['ds'], errors='coerce').fillna(pd.Timestamp.now()).dt.strftime('%Y-%m-%dT%H:%M:%S'))
    else:
        dados_treino = pd.DataFrame(columns=dados_teste.columns) # Caso não precise ter os dados de treino e agilizar o treinamento dos outros modelos. Posso fazer so os testes depois.
         
    # Prepara o dataframe final unindo treino, validação e teste
    colunas = [
        'treino_id',    # Identificador do treino
        'unique_id',    # Identificador de cada série
        'ds',           # Data de cada registro
        'y',            # Valor real
        'y_pred',       # Valor predito pelo modelo
        'diferença_%',  # Valor da diferença percentual entre o valor predito e o real
        'flag',         # Indica se foi usado em treino, validação ou teste
        'dataset',      # Nome do dataset usado
        'modelo',       # Nome do algoritmo (LSTM, Random Forest, etc)
        'comentario',   # Anotações adicionais (pode ser vazio)
        'data_treino'   # Data que o modelo foi treinado
    ]

    # Verificar existência do arquivo
    # Se o dataframe com os dados de previsão não existir, ele será criado
    if not os.path.exists(f"{csv_dir}"):
        # Criar DataFrame vazio com as colunas especificadas
        df = pd.DataFrame(columns=colunas)

        # Salvar o DataFrame vazio como CSV
        df.to_csv(f"{csv_dir}", index=False)
        print(f"Arquivo {csv_dir} criado com DataFrame vazio.")
    else:
        df = pd.read_csv(f"{csv_dir}")
        df['ds'] = (pd.to_datetime(df['ds'], errors='coerce').fillna(pd.Timestamp.now()).dt.strftime('%Y-%m-%dT%H:%M:%S'))
        print(f"Arquivo {csv_dir} já existe. Nenhuma ação necessária.")

    
    # Concatena os dados de treino, validação e teste em um único DataFrame. Considerando a posibilidade de val_ds ser None
    if val_ds is not None:
        # Concatena os dados de treino, validação e teste
        dados_completos = pd.concat([dados_treino, dados_validacao, dados_teste], ignore_index=True)
    else:
        # Concatena apenas os dados de treino e teste
        dados_completos = pd.concat([dados_treino, dados_teste], ignore_index=True)
    

    dados_completos['treino_id'] = treino_id  # Identificador do treino
    dados_completos['dataset'] = nome_dataset  # Nome do dataset usado
    dados_completos['modelo'] = "LSTM"
    dados_completos['data_treino'] = datetime.now().strftime('%Y-%m-%dT%H:%M:%S')


    # Adiciona o comentário detalhado sobre o treinamento
    if val_ds is not None:
        if val_ds['ds'].dt.year.min() == val_ds['ds'].dt.year.max():
            periodo_validacao = f"validado com os dados de {val_ds['ds'].dt.year.min()}"
        else:
            periodo_validacao = f"validado com os dados de {val_ds['ds'].dt.year.min()} a {val_ds['ds'].dt.year.max()}"
    else:
        periodo_validacao = "sem validação"

    if test_ds['ds'].dt.year.min() == test_ds['ds'].dt.year.max():
        periodo_teste = f"testado com os dados de {test_ds['ds'].dt.year.min()}"
    else:
        periodo_teste = f"testado com os dados de {test_ds['ds'].dt.year.min()} a {test_ds['ds'].dt.year.max()}"

    dados_completos['comentario'] =f"""{local}
    Modelo LSTM treinado com dados de {train_ds['ds'].dt.year.min()} a {train_ds['ds'].dt.year.max()}, {periodo_validacao} e {periodo_teste}.
    Para esse treinamento foi utilizado o dataset {nome_dataset}.
    {comentario}

    O modelo foi treinado com os seguintes Hiperparâmetros:
    input_size: {hiperparametros['input_size']} 
    h: {hiperparametros['h']} (horizonte de previsão)

    encoder_n_layers = {hiperparametros['encoder_n_layers']}
    learning_rate: {hiperparametros['learning_rate']}
    encoder_hidden_size: {hiperparametros['encoder_hidden_size']}
    decoder_layers: {hiperparametros['decoder_layers']}
    decoder_hidden_size: {hiperparametros['decoder_hidden_size']}
    batch_size: {hiperparametros['batch_size']}
    dropout: {hiperparametros['dropout']}
    weight_decay: {hiperparametros['weight_decay']}
    steps: {hiperparametros['steps']}
    """

    # Reordenar as colunas
    nova_ordem_colunas = ['treino_id', 'unique_id', 'ds', 'y', 'y_pred', 'diferença_%', 'flag', 'dataset', 'modelo', 'comentario', 'data_treino']
    dados_completos = dados_completos[nova_ordem_colunas]
    # Inclui as novas previsões no arquivo existente
    df_final = pd.concat([df, dados_completos], ignore_index=True)
    # Salva o DataFrame atualizado de volta ao CSV
    df_final.to_csv(f"{csv_dir}", index=False)
    clear_output()
    print(string_saidas)
    print(f"Previsões salvas no arquivo {csv_dir}")

In [5]:
# Calcula a diferença percentual absoluta entre previsão e valor real, ela é usada durante para gerar o dataframe com as previsões.
# E como o dataframe de previsões inclui os dados de teste e validação, ela precisa lidar com o periodo de treino que não tem previsão,
# os anos referentes a janela de contexto e nesse caso retorna "-".
def calcula_diferenca_pct(row):
    if row['y_pred'] == "-" or row['y_pred'] is None:
        return "-"
    try:
        y_pred_float = float(row['y_pred'])
        y_real = float(row['y'])
        # Evita divisão por zero
        if y_real == 0:
            return "-"
        # Calcula diferença percentual absoluta
        diff_pct = abs((y_pred_float - y_real) / y_real) * 100
        return round(diff_pct, 2)
    except:
        return "-"

In [6]:
def get_hyperparameters_from_yaml(yaml_path: str) -> dict:
    """
    Lê um arquivo hparams.yaml, extrai um conjunto específico de hiperparâmetros
    e os retorna em um dicionário.

    Esta função lida com os arquivos hparams.yaml gerados pelo PyTorch Lightning,
    que podem conter tags de objetos Python e requerem o uso de 'UnsafeLoader'.

    Args:
        yaml_path (str): O caminho para o arquivo hparams.yaml.

    Returns:
        dict: Um dicionário contendo os hiperparâmetros extraídos.
              Ex: {'learning_rate': 0.001, 'batch_size': 32, ...}
        
        None: Retorna None se o arquivo não for encontrado, se houver um erro de
              leitura do YAML ou se uma chave esperada não for encontrada.
    """
    # Tenta ler o arquivo YAML
    try:
        with open(yaml_path, 'r', encoding='utf-8') as f:
            config_data = yaml.load(f, Loader=yaml.UnsafeLoader)
    except FileNotFoundError:
        print(f"ERRO: Arquivo não encontrado no caminho: {yaml_path}")
        return None
    except yaml.YAMLError as e:
        print(f"ERRO: Ocorreu um problema ao processar o arquivo YAML: {e}")
        return None

    # Tenta extrair os parâmetros específicos
    try:
        hyperparameters = {
            'encoder_n_layers': config_data['encoder_n_layers'],
            'learning_rate': config_data['learning_rate'],
            'input_size': config_data['input_size'],
            'encoder_hidden_size': config_data['encoder_hidden_size'],
            'decoder_layers': config_data['decoder_layers'],
            'decoder_hidden_size': config_data['decoder_hidden_size'],
            'batch_size': config_data['batch_size'],
            'dropout': config_data['encoder_dropout'],  # Mapeado de 'encoder_dropout'
            'weight_decay': config_data['optimizer_kwargs']['weight_decay'], # Mapeado de um dicionário aninhado
            'steps': config_data['max_steps'] # Mapeado de 'max_steps'
        }
        return hyperparameters
    except KeyError as e:
        print(f"ERRO: A chave de hiperparâmetro esperada {e} não foi encontrada no arquivo.")
        return None

### Resumo das Métricas WMAPE

O objetivo de cada função é avaliar o desempenho do seu modelo de previsão sob uma ótativa diferente.

#### 1. Média Simples dos WMAPEs (`calcular_media_wmape_simples`)

* **Como Funciona:**
    1.  O modelo olha para cada município de forma isolada.
    2.  Calcula o erro percentual (WMAPE) apenas para aquele município.
    3.  Depois de fazer isso para todos os municípios, ele simplesmente tira a **média aritmética** de todos esses erros individuais.

* **Principal Objetivo:** Responder à pergunta: **"Em média, qual é o desempenho do meu modelo em cada localidade, tratando todas como igualmente importantes?"**
    * É uma visão "democrática". Um erro de 20% em um município pequeno tem o mesmo peso na nota final que um erro de 20% em um município gigante. Use esta métrica para garantir que seu modelo seja consistente e não tenha um desempenho muito ruim em localidades específicas, independentemente do tamanho delas.

#### 2. WMAPE da Soma Total ou Agregado (`calcular_wmape_agregado`)

* **Como Funciona:**
    1.  Primeiro, ele ignora as divisões por município.
    2.  Soma **toda a produção real** de todos os municípios para obter a safra total real do estado.
    3.  Soma **toda a produção prevista** de todos os municípios para obter a safra total prevista.
    4.  Calcula um único WMAPE comparando esses dois totais (Total Real vs. Total Previsto).

* **Principal Objetivo:** Responder à pergunta: **"No final, o quão perto eu cheguei de acertar a safra TOTAL do estado de Minas Gerais?"**
    * Esta é a visão do "resultado final". Ela foca no número mais importante para o negócio. Esta métrica permite que erros se cancelem (prever a mais em um município pode compensar prever a menos em outro), refletindo a precisão da previsão no nível mais alto.

#### 3. Média Ponderada dos WMAPEs (`calcular_wmape_ponderado`)

* **Como Funciona:**
    1.  Começa como a primeira métrica: calcula o erro (WMAPE) individualmente para cada município.
    2.  Porém, antes de calcular a média final, ele atribui um **"peso de importância"** a cada município, baseado no seu volume de produção.
    3.  A nota final é uma **média ponderada**, onde os erros dos municípios que produzem mais têm um impacto muito maior no resultado.

* **Principal Objetivo:** Responder à pergunta: **"Qual é o desempenho médio do meu modelo, considerando que um erro nos municípios grandes é muito mais prejudicial que um erro nos pequenos?"**
    * É uma visão de "impacto no negócio". Ela mede a performance média, mas de uma forma que reflete a importância econômica de cada localidade. É um excelente meio-termo entre a visão puramente local (Métrica 1) и a visão puramente agregada (Métrica 2).

In [7]:
# -----------------------------------------------------------------------------
# Função base (helper) que calcula o WMAPE para um par de vetores
# -----------------------------------------------------------------------------
def wmape_base(actual: np.ndarray, predicted: np.ndarray) -> float:
    """Calcula o WMAPE para um conjunto de valores reais e previstos."""
    # Garante que não haverá divisão por zero. Se a soma real for 0, o erro é 0.
    sum_actual = np.sum(np.abs(actual))
    if sum_actual == 0:
        return 0.0
    return np.sum(np.abs(predicted - actual)) / sum_actual

# -----------------------------------------------------------------------------
# Métrica 1: Média Simples dos WMAPEs por Município
# -----------------------------------------------------------------------------
def calcular_media_wmape_simples(df: pd.DataFrame) -> float:
    """
    Calcula o WMAPE para cada município (unique_id) individualmente e
    retorna a média simples desses WMAPEs.
    Trata todos os municípios como igualmente importantes.
    """
    # Agrupa por município e calcula o WMAPE para cada um
    wmapes_individuais = df.groupby('unique_id')[['y', 'LSTM']].apply(
        lambda g: wmape_base(g['y'].values, g['LSTM'].values)
    )
    
    # Retorna a média dos WMAPEs calculados
    return wmapes_individuais.mean()

# -----------------------------------------------------------------------------
# Métrica 2: WMAPE da Soma Total (Agregado) - O que você já usa
# -----------------------------------------------------------------------------
def calcular_wmape_agregado(df: pd.DataFrame) -> float:
    """
    Soma todas as previsões e todos os valores reais do dataframe inteiro
    e calcula um único WMAPE sobre esses totais.
    Foca na acurácia da previsão agregada (total do estado).
    """
    actual_total = df['y'].values
    predicted_total = df['LSTM'].values
    
    return wmape_base(actual_total, predicted_total)

# -----------------------------------------------------------------------------
# Métrica 3: Média Ponderada dos WMAPEs por Município
# -----------------------------------------------------------------------------
def calcular_wmape_ponderado(df: pd.DataFrame) -> float:
    """
    Calcula o WMAPE para cada município e depois faz uma média ponderada,
    onde o peso de cada município é sua participação na produção total.
    """
    # 1. Calcula o WMAPE para cada município
    wmapes_individuais = df.groupby('unique_id')[['y', 'LSTM']].apply(
        lambda g: wmape_base(g['y'].values, g['LSTM'].values)
    )
    
    # 2. Calcula o peso de cada município (sua produção total / produção geral)
    producao_por_municipio = df.groupby('unique_id')['y'].sum()
    producao_total = df['y'].sum()
    
    # Evita divisão por zero se a produção total for 0
    if producao_total == 0:
        return 0.0
        
    pesos = producao_por_municipio / producao_total
    
    # 3. Multiplica os WMAPEs pelos pesos e soma o resultado
    wmape_final_ponderado = np.sum(wmapes_individuais * pesos)
    
    return wmape_final_ponderado

In [8]:
def evaluate_simple_forecast(
    model,
    train_df: pd.DataFrame,
    test_df: pd.DataFrame,
    model_name: str = 'LSTM',  
    split: str ="",
    string_saidas: str = "",
    inteiro: bool = False,
    log_transformed: bool = True
) -> pd.DataFrame:
    """
    Executa avaliação de previsão considerando o nome do modelo dinâmico.
    """
    
    # Limpeza e Logs (conforme seu original)
    if string_saidas:
        clear_output()
        print(string_saidas)

    print(f"{split} Iniciando a previsão com o modelo: {model_name}...")

    # --- 1. Preparação dos Dados ---
    history_df = train_df.copy()
    future_data_to_evaluate = test_df.copy()

    # Conversão de datas
    history_df['ds'] = pd.to_datetime(history_df['ds'])
    future_data_to_evaluate['ds'] = pd.to_datetime(future_data_to_evaluate['ds'])

    # Filtra histórico para evitar vazamento de dados
    cutoff = future_data_to_evaluate['ds'].min()
    history_df = history_df[history_df['ds'] < cutoff]
  
    # Prepara df futuro para predict (sem y)
    futr_df = future_data_to_evaluate.drop(columns=["y"], errors='ignore')

    # --- 2. Geração da Previsão ---
    forecasts_df = model.predict(
        df=history_df,
        futr_df=futr_df
    )
    
    # Verificação de segurança: A coluna do modelo existe?
    if model_name not in forecasts_df.columns:
        # Tenta fallback inteligente ou erro
        cols_disponiveis = [c for c in forecasts_df.columns if c not in ['unique_id', 'ds']]
        raise ValueError(f"A coluna '{model_name}' não foi encontrada na previsão. Colunas disponíveis: {cols_disponiveis}")

    print("Previsão concluída. Combinando com dados reais...")

    # --- 3. Pós-processamento e Avaliação ---
    evaluation_df = forecasts_df.merge(
        future_data_to_evaluate[["unique_id", "ds", "y"]],
        on=["unique_id", "ds"],
        how="inner"
    )

    # Tratamento de Log (Reversão)
    if log_transformed:
        evaluation_df['y'] = np.expm1(evaluation_df['y'])
        evaluation_df[model_name] = np.expm1(evaluation_df[model_name]) # Usa model_name

    # Arredondamento
    if inteiro:
        evaluation_df[model_name] = evaluation_df[model_name].round().astype(int)
        evaluation_df['y'] = evaluation_df['y'].round().astype(int)

    # Cálculo da Diferença Percentual (passando o nome da coluna)
    evaluation_df = calcular_diferenca_percentual(evaluation_df, col_pred=model_name)

    print("Avaliação finalizada.")
    return evaluation_df


def calcular_diferenca_percentual(df: pd.DataFrame, col_pred: str = 'LSTM') -> pd.DataFrame:
    """
    Calcula a diferença percentual dinamicamente baseada na coluna predita.
    """
    df = df.copy()
    
    # Fórmula: ((Predito - Real) / Real) * 100
    # Adicionei tratamento para divisão por zero com np.where
    df['diferença_%'] = np.where(
        df['y'] == 0, 
        0,  # Se real for 0, define erro como 0 para evitar inf (ajuste conforme regra de negócio)
        ((df[col_pred] - df['y']) / df['y']) * 100
    )
    
    return df

In [9]:
# ===================================================================
# FÁBRICA DE CALLBACKS
# Responsabilidade: Criar uma função de callback configurada com o caminho correto de onde salvar o modelo.
# ===================================================================
def create_save_best_model_callback(destination_path: Path):
    """
    Esta função é uma "fábrica": ela recebe o caminho de destino e retorna
    a função de callback que o Optuna vai usar.
    """
    
    # Esta é a função que o Optuna realmente vai chamar.
    # Note que ela tem acesso à variável 'destination_path' da função externa.
    # ===================================================================
    # FUNÇÃO CALLBACK (RESPONSABILIDADE: SALVAR O MELHOR MODELO)
    # ===================================================================
    def callback(study: optuna.study.Study, trial: optuna.trial.FrozenTrial):
        """
        Callback que é acionado após cada trial para salvar o melhor modelo.
        """
        if study.best_trial.number == trial.number:
            print(f"\n✨ Novo melhor trial encontrado: #{trial.number} com Score: {trial.value:.4f}")
            
            source_path_str = trial.user_attrs.get("artifact_path")
            prediction_df_dict = trial.user_attrs.get("prediction_df")

            if not source_path_str or not prediction_df_dict:
                print("   -> AVISO: Não foi possível encontrar os artefatos para salvar.")
                return

            source_path = Path(source_path_str)
            # *** A MUDANÇA CRÍTICA ESTÁ AQUI ***
            # Usa o caminho que foi "injetado" quando a função foi criada
            final_destination = destination_path 

            print(f"   -> Salvando artefatos de '{source_path}' para '{final_destination}'...")

            if final_destination.exists():
                shutil.rmtree(final_destination)
            final_destination.mkdir(parents=True, exist_ok=True) # Adicionado exist_ok=True por segurança

            shutil.copytree(source_path, final_destination, dirs_exist_ok=True)
            
            prediction_df = pd.DataFrame(prediction_df_dict)
            prediction_df.to_excel(final_destination / "valores_predicao.xlsx", index=False)
            
            print("   -> Modelo e artefatos salvos com sucesso!")

    # A fábrica retorna a função de callback configurada
    return callback

In [10]:
from typing import Tuple
def filtrar_datasets_por_integridade(
    train_df: pd.DataFrame, 
    val_df: pd.DataFrame, 
    test_df: pd.DataFrame
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Filtra os datasets de treino, validação e teste para manter apenas os
    municípios ('unique_id') que possuem dados completos no período de validação.

    Args:
        train_df (pd.DataFrame): DataFrame de treino.
        val_df (pd.DataFrame): DataFrame de validação. Usado como referência para a verificação.
        test_df (pd.DataFrame): DataFrame de teste.

    Returns:
        Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]: Uma tupla contendo os 
        DataFrames de treino, validação e teste devidamente filtrados.
    """
    print("--- Iniciando verificação de integridade dos dados ---")
    
    # Usa o DataFrame de validação como referência para a integridade.
    # 1. Conta o número de timestamps únicos que cada município DEVERIA ter.
    timestamps_esperados = len(val_df['ds'].unique())
    if timestamps_esperados == 0:
        print(f"{CoresTerminal.VERMELHO}ALERTA: O conjunto de validação está vazio. Nenhum filtro será aplicado.{CoresTerminal.FIM}")
        return train_df, val_df, test_df

    # 2. Conta quantos timestamps únicos cada município realmente POSSUI.
    contagem_por_id = val_df.groupby('unique_id')['ds'].nunique()
    
    # 3. Identifica os municípios com dados completos.
    ids_completos = contagem_por_id[contagem_por_id == timestamps_esperados].index
    
    # 4. Compara com o total de municípios para ver se a filtragem é necessária.
    ids_originais = val_df['unique_id'].nunique()
    
    if len(ids_completos) < ids_originais:
        total_removido = ids_originais - len(ids_completos)
        
        print(f"{CoresTerminal.VERMELHO}"
              "----------------------------------------------------------------------\n"
              "ALERTA: Inconsistência de dados encontrada.\n"
              f"Foram encontrados {total_removido} de {ids_originais} municípios com dados INCOMPLETOS no período de validação.\n"
              "Todos os datasets serão filtrados para manter apenas os municípios com dados completos.\n"
              f"----------------------------------------------------------------------{CoresTerminal.FIM}")
        
        # Filtra TODOS os dataframes para manter apenas os municípios com dados completos.
        # O uso de .copy() evita o SettingWithCopyWarning do pandas.
        train_filtrado = train_df[train_df['unique_id'].isin(ids_completos)].copy()
        val_filtrado = val_df[val_df['unique_id'].isin(ids_completos)].copy()
        test_filtrado = test_df[test_df['unique_id'].isin(ids_completos)].copy()
        
        # Se após a filtragem não sobrar nenhum dado, interrompe a execução.
        if val_filtrado.empty:
            print(f"{CoresTerminal.VERMELHO}ALERTA: Após a filtragem, não restaram dados válidos. Retornando DataFrames vazios.{CoresTerminal.FIM}")
            return train_filtrado, val_filtrado, test_filtrado
        
        print(f"Filtragem concluída. {len(ids_completos)} municípios mantidos.")
        return train_filtrado, val_filtrado, test_filtrado
        
    else:
        print(f"{CoresTerminal.VERDE}Verificação concluída. Todos os {ids_originais} municípios possuem dados completos no período de validação.{CoresTerminal.FIM}")
        return train_df, val_df, test_df

In [11]:
def get_dataset(dataset_file):
    """
    Carrega, processa e limpa o dataset, garantindo que todas as séries temporais
    para cada 'unique_id' estejam completas no intervalo de datas do dataset.
    """
    dataset = pd.read_csv(dataset_file)
    # --- Aplica transformação logarítmica ---
    log_cols = [
        'Área colhida (Hectares)',
        'target',
        'precomediocafe', # a partir de novembro 2025
        'Área destinada à colheita (Hectares)'
    ]
    existing_log_cols = [col for col in log_cols if col in dataset.columns]
    if existing_log_cols:
        dataset[existing_log_cols] = dataset[existing_log_cols].apply(lambda x: np.log1p(x))

    # --- Renomeação e formatação para o padrão NeuralForecast ---
    dataset = dataset.rename(columns={
        "municipio": "unique_id",
        #"Municipio": "unique_id",
        "ano": "ds",
        #"Ano": "ds",
        "target": "y"
    })
    # Ordenar por unique_id e ds (ano)
    dataset = dataset.sort_values(by=["unique_id", "ds"]).reset_index(drop=True)
    dataset['ds'] = pd.to_datetime(dataset['ds'].astype(str) + '-12-31')
    return dataset

# Treinamento

In [ ]:
dataset_path = "Dataset/V43"

# sempre coloque a data no inicio do nome.
Local_treino =  "teste_2020_2025_por_cluster" # Nome do arquivo final com todas as predições
nome_dataset = "V43"
anos_validacao = [2019, 2020, 2021, 2022, 2023, 2024] #  Anos usados para validação (caso for validar com mais de um ano, coloque apenas o ano de inicio, pois o ano de teste sera o ano_val +h)

comentario = """O dataset foi dividido em 5 Clusters.
Nesta versão cadacluster tem seu proprio conjunto de features.
Nesta  versão usamos o dataset V43 para testar o modelo entre os anos de 2016-2025.
Foi usado a partir de 2016, para que pudese usar pelo menos 2 ano de contexto durante o treinamento e 1 de validação."""

In [13]:
# Lista de parametros usados no modelo
hiperparametros = { 
    'h': 1,
    'input_size': -1, # Use -1 para janela de contexto máxima
    'batch_size': 32,
    'dropout': 0.3,
    'encoder_n_layers': 4,
    'learning_rate': 0.00013034723280377263,
    'encoder_hidden_size': 192,
    'decoder_layers': 1,
    'decoder_hidden_size': 64,
    'weight_decay': 0.0001,
    'steps': 1000
}

In [14]:
for ano_val in anos_validacao:
    ano_teste = ano_val + hiperparametros['h'] # Define o ano de teste com base no horizonte. 
    dataset_path_ano = f"{dataset_path}/{ano_teste}"
    lista_datasets = os.listdir(dataset_path_ano)
    for dataset_file in lista_datasets:
        
        treino_id = f"{dataset_file[:-4]}_{ano_teste}"
        dataset = get_dataset(f"{dataset_path_ano}/{dataset_file}")   

        # --- Coleta a lista variaveis exogenas ---
        exog_list = [col for col in dataset.columns.tolist() if col not in ["ds", "y", "unique_id"]]

        # --- Preparação do dataset de treino, validação e teste ---
        ano_teste = ano_val + hiperparametros['h'] # Define o ano de teste com base no horizonte. 
        train_ds = dataset[dataset['ds'].dt.year < ano_val].copy()
        val_ds = dataset[(dataset['ds'].dt.year >= ano_val) & (dataset['ds'].dt.year < ano_teste)].copy()
        test_ds = dataset[dataset['ds'].dt.year >= ano_teste].copy()

        # --- Aplica a função de filtragem para garantir a consistência ---
        clear_output()
        print(f"== Iniciando processamento para o dataset: {dataset_file} com validação em {ano_val} e teste em {ano_teste} ===\n")
        train_ds, val_ds, test_ds = filtrar_datasets_por_integridade(
            train_df=train_ds,
            val_df=val_ds,
            test_df=test_ds
        )
        print(f"Dados de Treino entre os anos de {train_ds['ds'].dt.year.min()} e {train_ds['ds'].dt.year.max()}: {len(train_ds)} registros")
        print(f"Dados de Validação entre os anos de {val_ds['ds'].dt.year.min() if val_ds is not None else 'N/A'} e {val_ds['ds'].dt.year.max() if val_ds is not None else 'N/A'}: {len(val_ds) if val_ds is not None else 0} registros")
        print(f"Dados de Teste entre os anos de {test_ds['ds'].dt.year.min()} e {test_ds['ds'].dt.year.max()}: {len(test_ds)} registros") 


        # -- Define o tamanho da janela de contexto (input_size) ---
        if hiperparametros['input_size'] == -1:
            hiperparametros['input_size'] = len(train_ds['ds'].unique()) - hiperparametros['h'] # Define o tamanho da janela de contexto como o total de anos de treino menos o horizonte

        print(f"\nTamanho da janela de contexto (input_size): {hiperparametros['input_size']}.\n")

        model = LSTM(
            h=hiperparametros['h'],
            input_size= hiperparametros['input_size'],
            batch_size=hiperparametros['batch_size'],
            scaler_type="revin",
            encoder_dropout=hiperparametros['dropout'],
            encoder_n_layers=hiperparametros['encoder_n_layers'],
            encoder_hidden_size=hiperparametros['encoder_hidden_size'],
            decoder_layers=hiperparametros['decoder_layers'],
            decoder_hidden_size=hiperparametros['decoder_hidden_size'],
            futr_exog_list=exog_list,
            learning_rate=hiperparametros['learning_rate'],
            max_steps=hiperparametros['steps'],
            loss=HuberLoss(delta=1.0),
            optimizer=AdamW,
            optimizer_kwargs={"weight_decay": hiperparametros['weight_decay']},
            lr_scheduler=StepLR,
            lr_scheduler_kwargs={"step_size": int(hiperparametros['steps'] * 0.5), "gamma": 0.1},
            random_seed=42
        )

        nf = NeuralForecast(models=[model], freq="YE")
        nf.fit(df=train_ds)

        # --- Validando o modelo ---
        combined_df = evaluate_simple_forecast(
            model=nf,
            train_df=train_ds,
            test_df=val_ds
        )

        actual = combined_df["y"]
        predicted = combined_df["LSTM"]

        # Calcula as três versões da métrica
        score_media_simples = calcular_media_wmape_simples(combined_df)
        score_rmse = root_mean_squared_error(actual, predicted)
    
        # Imprime o score de validação
        print(f"\n\n --- Resultados da Validação ---")
        print(f"  - WMAPE: {score_media_simples:.4f}")
        print(f"  - RMSE:  {score_rmse:.4f}\n\n")

        # --- Salvando o modelo ---        
        local_save = f"./Treinos/{Local_treino}/Modelos/{treino_id}_({ano_teste})"
        csv_dir = f"./Treinos/{Local_treino}/{Local_treino}.csv"
        nf.save(
            path= local_save,
            overwrite=True)
        print(f"Modelo salvo em: {local_save}\n")
        # --- Avaliação ---
        teste_modelo(local_save, csv_dir, dataset, train_ds, val_ds, test_ds, comentario, nome_dataset, h=hiperparametros['h'])


Previsões salvas no arquivo ./Treinos/teste_2020_2025_por_cluster/teste_2020_2025_por_cluster.csv


In [15]:
print("Resultado agregado dos clusters:\n")

df_atual = pd.read_csv(csv_dir)
df_atual.rename(columns={"unique_id": "Municipio", "ds": "Ano"}, inplace=True)
df_atual['Ano'] = pd.to_datetime(df_atual['Ano']).dt.year

df_referencia = pd.read_csv("Treinos/Modelo_Altitude_Ajustado/modelo_altitude_ajustado.csv")
df_referencia.rename(columns={"unique_id": "Municipio", "ds": "Ano"}, inplace=True)
df_referencia['Ano'] = pd.to_datetime(df_referencia['Ano']).dt.year

area_colhida = pd.read_csv("Dataset/Area_colhida_V2.csv")

df_atual = df_atual[df_atual['flag'] == 'teste'].copy()
df_referencia = df_referencia[df_referencia['flag'] == 'teste'].copy()

# Converter para numérico
for col in ["y", "y_pred"]:
    df_atual[col] = pd.to_numeric(df_atual[col], errors='coerce')
    df_referencia[col] = pd.to_numeric(df_referencia[col], errors='coerce')

# Merge com área colhida
df_merged = pd.merge(
    df_atual[['treino_id', 'Municipio', 'Ano', 'y', 'y_pred']],
    area_colhida,
    on=['Municipio', 'Ano'],
    how='left'
)

# Conversão para produção total em sacas
saca_kg = 60
area_hectares = df_merged['Area']

df_merged["y"] = (df_merged['y'] * area_hectares * 1000) / saca_kg
df_merged["y_pred"] = (df_merged['y_pred'] * area_hectares * 1000) / saca_kg

# Converter também o modelo de referência
df_referencia = pd.merge(
    df_referencia[['Municipio', 'Ano', 'y_pred']],
    area_colhida,
    on=['Municipio', 'Ano'],
    how='left'
)

df_referencia["y_pred"] = (df_referencia['y_pred'] * df_referencia['Area'] * 1000) / saca_kg

# Loop por ano
for ano in sorted(df_merged['Ano'].unique()):
    real = df_merged[df_merged['Ano'] == ano]['y'].sum().round(3)
    previsto = df_merged[df_merged['Ano'] == ano]['y_pred'].sum().round(3)
    previsto_ref = df_referencia[df_referencia['Ano'] == ano]['y_pred'].sum().round(3)

    print(f"{ano}: Real: {real}, Previsto: {previsto} (Referência: {previsto_ref})")

Resultado agregado dos clusters:

2020: Real: 33996366.667, Previsto: 35426852.851 (Referência: 30568151.922)
2021: Real: 22345000.0, Previsto: 27933555.424 (Referência: 31813822.776)
2022: Real: 22818800.0, Previsto: 33752861.735 (Referência: 27571501.049)
2023: Real: 28483900.0, Previsto: 27825387.199 (Referência: 19462848.631)
2024: Real: 27650983.333, Previsto: 24864461.166 (Referência: 25181333.022)
2025: Real: 0.0, Previsto: 22058228.376 (Referência: 25560650.275)


# Inferencia pos treino


In [ ]:
def converter_colunas_para_minusculo(df):
    """
    Função que converte os nomes das colunas de um dataframe para minúsculas

    Parâmetros:
    - df: DataFrame que terá as colunas convertidas para minúsculas

    Retorna:
    - DataFrame com colunas em minúsculas
    """
    df_renomeado = df.rename(columns=lambda col: col.lower())
    return df_renomeado

In [ ]:
for i in range (0,200):
    local = f"Logs/trial_{i}/version_0"
    if not os.path.exists(local):
        os.makedirs(f"Logs/trial_{i}/version_0")

In [ ]:
df_final = pd.DataFrame()
ano = 2021
for cluster in range(5):   
    dataset = get_dataset(f"Dataset/V29 - Cluster 5/dataset_V29_cluster_{cluster}_metodo_1.csv") 
    model = NeuralForecast.load(path=f"Treinos/Antigos/Antigos_Antigos/Teste_Clusters/Modelos_antigos/IBGE - Cluster V5 Cluster {cluster} (2025)")

    #if ano == 2025:
    dataset = converter_colunas_para_minusculo(dataset)
        
    test_ds = dataset[dataset['ds'].dt.year == ano].copy()
    predictions = evaluate_simple_forecast(
        model=model,
        train_df=dataset,
        test_df=test_ds,
        split = "Teste"
    )

    dados_teste = predictions[['unique_id', 'ds', 'y', 'LSTM', 'diferença_%']].copy()
    dados_teste.columns = ['unique_id', 'ds', 'y', 'y_pred', 'diferença_%'] # Renomeando as colunas para o formato esperado
    dados_teste['flag'] = 'teste'
    dados_teste['cluster'] = cluster
    dados_teste['ds'] = pd.to_datetime(dados_teste['ds']).fillna(dados_teste['ds'])
    dados_teste['ds'] = (pd.to_datetime(dados_teste['ds'], errors='coerce').fillna(pd.Timestamp.now()).dt.strftime('%Y-%m-%dT%H:%M:%S'))
    df_final = pd.concat([df_final, dados_teste], ignore_index=True)

    
clear_output()

In [ ]:
df_area = pd.read_csv("Dataset/Area_colhida_V2.csv")
#df_area = pd.read_csv("Dataset/Area_colhida_projesao_2026.csv")
#df_area = pd.read_excel("Dataset/Projeções de áreas colhidas v2 - 2025 e 2026.xlsx")

df_area.rename(columns={"Área colhida (Hectares)": "Area"}, inplace=True)
for unique_id in df_final['unique_id'].unique():
    # Filtrar as linhas correspondentes no df_soma
    mask_final = df_final['unique_id'] == unique_id

    # Buscar área correspondente
    area_row = df_area[(df_area['Municipio'] == unique_id) & (df_area['Ano'] == ano)]
    
    if area_row.empty:
        print(f"Aviso: Área não encontrada para unique_id = {unique_id}")
        area_total = np.nan  # ou continue, ou defina um valor padrão
    else:
        area_total = area_row['Area'].iloc[0]

    # Aplicar a transformação SOMENTE às linhas desse unique_id
    if pd.notna(area_total):
        df_final.loc[mask_final, 'y_pred'] = (
            (df_final.loc[mask_final, 'y_pred'] * area_total) * 1000
        ) / 60
    else:
        df_final.loc[mask_final, 'y_pred'] = np.nan  # ou 0, dependendo da lógica

print(f"Ano: {ano} -> {df_final['y_pred'].sum()}")

In [ ]:
# Lista de parametros usados no modelo
hiperparametros = { 
    'h': 1,
    'input_size': 3, # Use -1 para janela de contexto máxima
    'batch_size': 32,
    'dropout': 0.3,
    'encoder_n_layers': 64,
    'learning_rate': 0.0001092083381402252,
    'encoder_hidden_size': 128,
    'decoder_layers': 2,
    'decoder_hidden_size': 128,
    'weight_decay': 0.0001,
    'steps': 200
}

In [ ]:
#cluster = 0
ano = 2021
#for ano in [2020, 2021, 2022, 2023, 2024]:
for cluster in range(5):
    dataset = get_dataset(f"Dataset/V29 - Cluster 5/dataset_V29_cluster_{cluster}_metodo_1.csv") 
    #dataset = converter_colunas_para_minusculo(dataset)

    local_modelo = f"Treinos/Antigos/Antigos_Antigos/Teste_Clusters/Modelos_antigos/IBGE - Cluster V5 Metodo 1 Modelo unico (2021)"
    csv_dir = "teste/IBGE_Cluster_V5_metodo_1_2021.csv"
    train_ds = dataset[dataset['ds'].dt.year < ano-1].copy()
    val_ds = dataset[dataset['ds'].dt.year == ano-1].copy()
    test_ds = dataset[dataset['ds'].dt.year == ano].copy()
    comentario = "dataset V29 com com as features escritas em minusculo. para padronizar de agora em diante."
    nome_dataset = "V29"
    treino_id = f"IBGE - Cluster V5 Metodo 1 Cluster {cluster} ({ano})"
    teste_modelo(local_modelo, csv_dir, dataset, train_ds, val_ds, test_ds, comentario, nome_dataset, h=hiperparametros['h'])

In [ ]:
df = pd.read_csv("teste/IBGE_Cluster_V5_metodo_1_2025.csv")
df_original = pd.read_csv("Treinoss/cluster_V5_Cluster_separados/cluster_V5_Cluster_separados.csv")

In [ ]:
df_novo = pd.concat([df_original, df], ignore_index=True)

In [ ]:
df_novo["treino_id"].unique()
df_novo.to_csv("Treinoss/cluster_V5_Cluster_separados/cluster_V5_Cluster_separados_2024.csv", index=False)